In [5]:
from nltk.tokenize.casual import casual_tokenize
import gensim
from gensim import corpora
import re
import pandas as pd
import numpy as np
from string import punctuation
from gensim.models import CoherenceModel
import os,sys

input_filepath="../../data/clean/"
filename="trisma2016_cleaned_text_by_lga.csv"

In [17]:

sys.path.append("../..")
from src.data import utils
nswdf_target = utils.load_target()
#target.columns23
nswdf_target=nswdf_target.set_index("LGA")

In [38]:
# Set up log to terminal
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.CRITICAL)
logging.getLogger().setLevel(logging.WARN)

In [6]:


tweetdf=pd.read_csv(os.path.join(input_filepath, filename),header=0)
tweetdf

,Unnamed: 0,lga,text,cleaned
0,0,Albury,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...,#2015bestnine #bestofnine2015 #byebye #2015 #h...
1,1,Armidale Regional,Tranquility in the Park. @ Moore Park Inn Bout...,tranquility in the park moore park inn boutiqu...
2,2,Ballina,Happy New Year to all my pals back home in the...,happy new year to all my pals back home in the...
3,3,Balranald,Crackin night on the river. https://t.co/lGYg4...,crackin night on the river just posted photo o...
4,4,Bathurst Regional,@ReneeYoungWWE Happy New Year to you &amp; you...,happy new year to you your fam as well miss mi...
5,5,Bayside,@tamasys @mattRan wtf is maths circus? @toddjt...,wtf is maths circus sounds like it must've bee...
6,6,Bega Valley,Suns out guns out. Coffee time. #crossfitgirls...,suns out guns out coffee time #crossfitgirls #...
7,7,Bellingen,More from the Promised Lands #normanheywood #l...,more from the promised lands #normanheywood #l...
8,8,Berrigan,@MoniqueAnnalyss hotttt 🔥🔥🔥 @MoniqueAnnalyss a...,hottt appreciate your hotness monique it's 201...
9,9,Blacktown,Ahhh Tumblr. I currently have a hammock in the...,ahhh tumblr currently have hammock in the livi...


In [7]:
#now split by various doc types and run LGA. DO i need to write out interim files? Not really, not yet

by_lga = tweetdf

In [24]:
def train_lda(traindf):
    toks = [x.split(" ") for x in traindf["cleaned"] ]

    dictionary = corpora.Dictionary(toks)
    dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000) 
    corpus = [dictionary.doc2bow(doc) for doc in toks]
    
    corpus = [dictionary.doc2bow(doc) for doc in toks]

    model = gensim.models.ldamodel.LdaModel(corpus, num_topics=300, id2word = dictionary, passes=50)
    
    return dictionary, model

In [37]:
model.save("lgaldamodel.pk")

In [97]:
model = gensim.models.ldamodel.LdaModel.load("lgaldamodel.pk")

In [22]:
def predict_lda(model, textdf, dictionary):
    toks = [x.split(" ") for x in textdf["cleaned"] ]
    corpus = [dictionary.doc2bow(doc) for doc in toks]
    inferred = model.inference(corpus)
    inferred = inferred[0]
    inferdf = pd.DataFrame(inferred)
    return inferdf

In [105]:
inferdf.to_csv(input_filepath+"ldabylgatrisma2016.csv")

In [49]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import probplot
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
import time
from collections import defaultdict

def kfold_cv(k, dataframe, target, num_of_crimes=6):
    lga_names = target.index
    start_col = target.columns.get_loc('Assault - domestic violence')
    kf = KFold(n_splits=5, shuffle=True)
    ldamodels = []
    rfmodels = defaultdict(list)
    rmse = defaultdict(list)
    r2 = defaultdict(list)
    mape = defaultdict(list)
    allpreds = target.copy()
    allpreds["preds"] = pd.Series()

    preds_start_col = len(allpreds.columns)
    for i in range(num_of_crimes):
        allpreds[target.columns[start_col+i]+"_preds"] = pd.Series()
    for train_index, test_index in kf.split(lga_names):
        start = time.time()
        train_lgas = lga_names[train_index]
        traindf = dataframe.loc[train_lgas]
        train_target = target.loc[train_lgas]
        
        test_lgas = lga_names[test_index]
        testdf = dataframe.loc[test_lgas]
        test_target = target.loc[test_lgas].copy()
        
        dic, ldamodel = train_lda(traindf)
        topics_train = predict_lda(ldamodel, traindf, dic)
        topics_test = predict_lda(ldamodel, testdf, dic)
        ldamodels.append(ldamodel)
        
        for crimenum in range(num_of_crimes):
            rfmodel = RandomForestRegressor()
            rfmodels[i].append(rfmodel)
            rfmodel.fit(topics_train, train_target.iloc[:,start_col+crimenum])
            #print(lga_loc)
            preds = rfmodel.predict(topics_test)

            allpreds.loc[test_lgas, target.columns[start_col+i]+"_preds"] = preds

            mape[i].append(utils.mape(preds, test_target.iloc[:,start_col+crimenum]))
            rmse[i].append(np.sqrt(mean_squared_error(preds, test_target.iloc[:,start_col+crimenum])))
            r2[i].append(r2_score(test_target.iloc[:,start_col+crimenum], preds))
        print(time.time() - start)
    #probplot(allpreds.iloc[:,start_col+crimenum] - allpreds.iloc[:,-1], dist='norm', plot=plt)
    #import pdb; pdb.set_trace()
    #plt.show()
    
    #mape.append(utils.mape(allpreds.iloc[:,start_col+crimenum], allpreds["preds"]))
    #rmse.append(np.sqrt(mean_squared_error(allpreds["preds"], allpreds.iloc[:,start_col+crimenum])))
    #r2.append(r2_score(allpreds.iloc[:,start_col+crimenum], allpreds["preds"]))
    return mape, rmse, r2, ldamodels, rfmodels, allpreds


output = kfold_cv(5, by_lga.set_index('lga'), nswdf_target)

/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


1608.4847910404205


/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


1680.7569568157196


/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


1639.933077096939


/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


5814.46976184845


/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


1451.5185227394104


In [53]:
allpreds = output[-1]
start_col = allpreds.columns.get_loc('Assault - domestic violence')
for i in range(6):
    colname = allpreds.columns[i]
    print(colname)
    print("mape:", utils.mape(allpreds.iloc[:,start_col+i], allpreds[colname+"_preds"]))
    #print("r2: ",outputs[i][3])
    #print("rmse: ",outputs[i][2])

Assault - domestic violence
mape: nan
Assault - non-domestic violence
mape: nan
Fraud
mape: nan
Sexual Offences
mape: nan
Malicious damage to property
mape: nan
Prohibited weapons offences
mape: 72.93779745470368


In [ ]:
for i in range(len(outputs)):
    print(nswdf_target.columns[i])
    print("mape:", outputs[i][1])
    print("r2: ",outputs[i][3])
    print("rmse: ",outputs[i][2])

In [102]:
imp = []
for i, output in enumerate(outputs):
    imp.append(np.zeros(300))
    for mod in output[3]:
        imp[-1] = imp[-1] + mod.feature_importances_
   

In [104]:
#for each target get most important topics
for i, importvec in enumerate(imp):
    print(nswdf_target.columns[i])
    top10 = np.argsort(importvec)[-10:]
    for j in top10:
        print(model.show_topic(j))
        

Assault - domestic violence
[('the', 0.02686654), ('to', 0.018986193), ('a', 0.016861826), ('of', 0.01193077), ('and', 0.011371217), ('in', 0.011186093), ('I', 0.009852552), ('for', 0.009473195), ('you', 0.0077860993), ('is', 0.00774266)]
[('at', 0.019395987), ('the', 0.015939591), ('NSW', 0.013259457), ('2016', 0.012779353), ('to', 0.010972102), ('a', 0.009879672), ('https://t.co/eZ1btQ7mTC', 0.00971782), ('Fire.', 0.009184353), ('#NSWRFS', 0.00885982), ('in', 0.0084838215)]
[('the', 0.024827428), ('@', 0.01946787), ('a', 0.018002212), ('in', 0.015646787), ('of', 0.014740214), ('to', 0.014446434), ('Blue', 0.01196658), ('Mountains', 0.008662943), ('I', 0.007946573), ('is', 0.007328172)]
[('today', 0.032852724), ('Rain', 0.03243294), ('Wind', 0.03239033), ('km/h', 0.032248646), ('Temperature', 0.032243315), ('Humidity', 0.0322371), ('mm.', 0.032234233), ('Barometer', 0.03223272), ('hPa,', 0.032231964), ('°C.', 0.032231636)]
[('the', 0.029153554), ('to', 0.01687482), ('a', 0.016309101),

[('the', 0.025515929), ('to', 0.021333946), ('a', 0.019739747), ('I', 0.016325584), ('in', 0.013463509), ('of', 0.012438776), ('and', 0.010472717), ('for', 0.009376953), ('is', 0.008640599), ('on', 0.0079719005)]
[('the', 0.025640914), ('to', 0.020387407), ('a', 0.017593753), ('I', 0.0139094535), ('and', 0.012350896), ('of', 0.011207191), ('in', 0.010694461), ('is', 0.010645284), ('for', 0.009696147), ('you', 0.008474594)]
[('the', 0.021310462), ('of', 0.021049976), ('@ellymelly', 0.017239274), ('a', 0.015415564), ('to', 0.011723669), ('@stiffleaf', 0.010749276), ('and', 0.008369316), ('@PoorFrankRaw', 0.008252378), ('in', 0.007613188), ('for', 0.0066999123)]
[('the', 0.02947543), ('a', 0.022736609), ('to', 0.021818679), ('I', 0.021577803), ('is', 0.0126913125), ('of', 0.01257679), ('you', 0.010175761), ('and', 0.010058351), ('in', 0.009887709), ('it', 0.009195653)]
Malicious damage to property
[('today', 0.032852724), ('Rain', 0.03243294), ('Wind', 0.03239033), ('km/h', 0.032248646), 

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
x = range(start, stop, step)
plt.plot(x, coh)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()